# Resources

In [1]:
import numpy as np
import pygmo as pg
import pygmo_plugins_nonfree as pg7
import PyKEP as pk
import matplotlib.pyplot as plt

#%matplotlib
#%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

/usr/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: to-Python converter for pagmo::population already registered; second conversion method ignored.
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: to-Python converter for pagmo::archipelago already registered; second conversion method ignored.
  return f(*args, **kwds)


# Algorithm

In [2]:
# algorithm
uda = pg7.snopt7(True, "/usr/lib/libsnopt7_c.so")
uda.set_integer_option("Major iterations limit", 4000)
uda.set_integer_option("Iterations limit", 40000)
uda.set_numeric_option("Major optimality tolerance", 1e-3)
uda.set_numeric_option("Major feasibility tolerance", 1e-8)
algo = pg.algorithm(uda)

# A priori

In [3]:
z0 = [
    2.97590873e+02, 8.43010991e-02, 6.26851629e+00,
    5.09737634e+01, -7.35789606e+01, -5.35749595e+00,
    9.89833613e+01, -1.07638347e-02, -1.41668668e+00,
    1.38626647e+01
]

p0 = pk.planet.jpl_lp("earth")
pf = pk.planet.jpl_lp("mars")

el0 = np.array(p0.osculating_elements(pk.epoch(0)))
elf = np.array(pf.osculating_elements(pk.epoch(0)))

mass = 1000
thrust = 0.3
isp = 2500

atol = 1e-10
rtol = 1e-10

Tlb = 100
Tub = 500

pi = 3.1459
Mlb = -2*pi
Mub = 2*pi

mu = pk.MU_SUN

udp = pk.trajopt.indirect_or2or(
    el0, elf, mass, thrust, isp,
    atol, rtol, Tlb, Tub, Mlb, Mub, Mlb, Mub,
    freemass=True, freetime=True, alpha=1, mu=pk.MU_SUN
)

udp.fitness(z0)

x0 = udp.leg.get_states()[0, 1:8]

T = z0[0]
Mf = z0[2]
l0 = z0[3:]

z0 = np.hstack(([T, Mf], l0))

# Indirect: Mass Optimal Point to Orbit
$$z = [T, M_f, \vec{\lambda}_0]$$

In [4]:
times = np.linspace(z0[0], 200, 30)

In [5]:
trajn = list()

for T in times:
    
    # constant time of flight
    Tlb, Tub = T, T
    
    # point to orbit problem
    udp = pk.trajopt.indirect_pt2or(
        x0, elf, mass, thrust, isp,
        atol, rtol, Tlb, Tub,
        Mlb, Mub, True, True,
        1, True, mu
    )
    
    # pygmo problem
    prob = pg.problem(udp)
    
    # constraint tolerance
    prob.c_tol = [1e-6]*udp.get_nec()
    
    # population
    pop = pg.population(prob, 0)
    
    # guess
    pop.push_back(z0)
    
    # optimise
    pop = algo.evolve(pop)
    
    # store solution
    z0 = pop.champion_x
    
    # store leg
    udp.fitness(pop.champion_x)
    trajn.append(udp.leg.get_states(atol, rtol))

RuntimeError: couldn't create a NumPy array: the 'PyArray_SimpleNew()' function failed

### plt.figure()

for traj in trajs:
    
    plt.plot()